# Alive & Thrive
## Viet Nam Data: Women Files - Preprocessing
## Prepared by Aaron Wise; aaron@a3di.dev
### Version: 25 May 2022

In [ ]:
# %load std_imports
from pathlib import Path

import numpy as np
import pandas as pd

import json
import pyreadstat

from std_utils import (
    read_file,
    run_quality_assurance,
    generate_HHID,
    merge_hh_hl_data,
    subset_df,
    standardize_col_names,
    standardize_col_values,
    create_elderly_hoh,
    save_merge,
    export_analyzed_data
)

pd.set_option("display.max_rows", 1500)
pd.set_option("display.max_columns", None)


In [ ]:
# %load women_imports.py
from women_analysis import (
    generate_str_replace_dict,
    create_anc_4_visits,
    create_anc_3_components,
    create_inst_delivery,
    create_caesarean_del,
    create_pnc_mother,
    create_low_bw,
    create_early_bf,
    create_mother_edu,
    subset_women_file
)

from aw_analytics import mean_wt, output_mean_table


### --- 2021 ---

In [ ]:
# Set parameters
country = 'VNM'
recode = 'women'

# -------------------------------------------------------------------
year = '2021'
# -------------------------------------------------------------------

In [ ]:
# Read file
df = read_file(country, year, recode)

# Create HHID to facilitate merge of HH and HL data
generate_HHID(df, country, year, recode)

# Run quality assurance
run_quality_assurance(df)

# Merge in HH and HL data
df = merge_hh_hl_data(df, country, year)

# Subset women file
df = subset_women_file(df, country, year)

#### Create Indicators

In [ ]:
# --- Create Mother edu [mother_edu] --- #
df = create_mother_edu(df, country, year)

In [ ]:
# --- ANC 4+ visits [anc_4_visits] --- #
df = create_anc_4_visits(df, country, year)

# --- ANC components [anc_3_components] --- #
df = create_anc_3_components(df, country, year)

# --- Institutional [inst_delivery] --- #
df = create_inst_delivery(df, country, year)

# --- Caesarean Delivery [caesarean_del] --- #
df = create_caesarean_del(df, country, year)

# Post-natal Health Check (mother) [pnc_mother]
df = create_pnc_mother(df, country, year)

# # --- Low birthweight [low_bw] --- #
# df = create_low_bw(df, country, year)

# --- Early Initiation BF [early_bf] --- #
df = create_early_bf(df, country, year)

### Export working variables

In [ ]:
export_analyzed_data(df, country, year, recode)

### --- 2014 ---

In [ ]:
# Set parameters
country = 'VNM'
recode = 'women'

# -------------------------------------------------------------------
year = '2014'
# -------------------------------------------------------------------

In [ ]:
# Read file
df = read_file(country, year, recode)

# Create HHID to facilitate merge of HH and HL data
generate_HHID(df, country, year, recode)

# Run quality assurance
run_quality_assurance(df)

# Merge in HH and HL data
df = merge_hh_hl_data(df, country, year)

# Subset women file
df = subset_women_file(df, country, year)

In [ ]:
# --- Create Mother edu [mother_edu] --- #
df = create_mother_edu(df, country, year)

In [ ]:
# --- ANC 4+ visits [anc_4_visits] --- #
df = create_anc_4_visits(df, country, year)

# --- ANC components [anc_3_components] --- #
df = create_anc_3_components(df, country, year)

# --- Institutional [inst_delivery] --- #
df = create_inst_delivery(df, country, year)

# --- Caesarean Delivery [caesarean_del] --- #
df = create_caesarean_del(df, country, year)

# Post-natal Health Check (mother) [pnc_mother]
df = create_pnc_mother(df, country, year)

# # --- Low birthweight [low_bw] --- #
# df = create_low_bw(df, country, year)

# --- Early Initiation BF [early_bf] --- #
df = create_early_bf(df, country, year)

#### Export working variables

In [ ]:
export_analyzed_data(df, country, year, recode)

### --- 2011 ---

In [ ]:
# Set parameters
country = 'VNM'
recode = 'women'

# -------------------------------------------------------------------
year = '2011'
# -------------------------------------------------------------------

In [ ]:
# Read file
df = read_file(country, year, recode)

# Create HHID to facilitate merge of HH and HL data
generate_HHID(df, country, year, recode)

# Run quality assurance
run_quality_assurance(df)

# Merge in HH and HL data
df = merge_hh_hl_data(df, country, year)

# Subset women file
df = subset_women_file(df, country, year)

#### Create Indicators

In [ ]:
# --- Create Mother edu [mother_edu] --- #
df = create_mother_edu(df, country, year)

In [ ]:
# --- ANC 4+ visits [anc_4_visits] --- #
df = create_anc_4_visits(df, country, year)

# --- ANC components [anc_3_components] --- #
df = create_anc_3_components(df, country, year)

# --- Institutional [inst_delivery] --- #
df = create_inst_delivery(df, country, year)

# --- Caesarean Delivery [caesarean_del] --- #
df = create_caesarean_del(df, country, year)

# # Post-natal Health Check (mother) [pnc_mother]
# df = create_pnc_mother(df, country, year)

# # --- Low birthweight [low_bw] --- #
# df = create_low_bw(df, country, year)

# --- Early Initiation BF [early_bf] --- #
df = create_early_bf(df, country, year)

### Export working variables

In [ ]:
export_analyzed_data(df, country, year, recode)

### --- 2006 ---

In [ ]:
# Set parameters
country = 'VNM'
recode = 'women'

# -------------------------------------------------------------------
year = '2006'
# -------------------------------------------------------------------

In [ ]:
# Read file
df = read_file(country, year, recode)

# Create HHID to facilitate merge of HH and HL data
generate_HHID(df, country, year, recode)

# Run quality assurance
run_quality_assurance(df)

# Merge in HH and HL data
df = merge_hh_hl_data(df, country, year)

# Subset women file
df = subset_women_file(df, country, year)

#### Create Indicators

In [ ]:
# --- Create Mother edu [mother_edu] --- #
df = create_mother_edu(df, country, year)

In [ ]:
# # --- ANC 4+ visits [anc_4_visits] --- #
# df = create_anc_4_visits(df, country, year)

# --- ANC components [anc_3_components] --- #
df = create_anc_3_components(df, country, year)

# --- Institutional [inst_delivery] --- #
df = create_inst_delivery(df, country, year)

# # --- Caesarean Delivery [caesarean_del] --- #
# df = create_caesarean_del(df, country, year)

# # Post-natal Health Check (mother) [pnc_mother]
# df = create_pnc_mother(df, country, year)

# # --- Low birthweight [low_bw] --- #
# df = create_low_bw(df, country, year)

# --- Early Initiation BF [early_bf] --- #
df = create_early_bf(df, country, year)

### Export working variables

In [ ]:
export_analyzed_data(df, country, year, recode)

### --- 2000 ---

In [ ]:
# Set parameters
country = 'VNM'
recode = 'women'

# -------------------------------------------------------------------
year = '2000'
# -------------------------------------------------------------------

In [ ]:
var = 'early_bf'
ind_vars = ['hh6']
wt = 'wmweight'

output_mean_table(df, var, ind_vars, wt)